<a href="https://colab.research.google.com/github/yo-ojin/the-system-for-LIS-patients/blob/main/video_fasterRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## OpenCV DNN 패키지를 이용한 Faster RCNN 기반 Object Detection 수행  
OpenCV를 이용하면 DNN API를 따오고, 이를 기반으로 (학습은 못하고...) Inference를 수행할 수 있다.  
아래에서 수행할 것은 object detection이고, 사용할 모델은 이전까지 이론적으로 학습한 Faster R-CNN이다.  
기본적으로 OpenCV 는 Tensorflow 에서 Pretrained 된 모델을 로드한다고하였다. 이미지, 모델을 순차적으로 다운받아보자.

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

### ✅ Tensorflow Pretrained Inference 모델, Config 파일 다운로드  
mkdir ./pretrained 명령어로 content/pretrained 디렉토리를 만들고 그 밑에 각각 pretrained 모델과 config 파일을 다운로드 받았다. 모델의 경우 .tar 로 압축되어 있는데, !tar 로 압축을 풀고 -C ./pretrained 로 이 밑에 풀 수 있다.  
다운로드 URL 은 다음과 같다.

[깃허브 다운로드 링크](https://velog.io/@kyungmin1029/CV-Faster-R-CNN-Object-Detection-%EC%8B%A4%EC%8A%B5#:~:text=%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C%20URL%20%EC%9D%80,%ED%99%98%EA%B2%BD%ED%8C%8C%EC%9D%BC%20%EB%AA%A8%EB%8D%B8)

[pretrained 모델](https://velog.io/@kyungmin1029/CV-Faster-R-CNN-Object-Detection-%EC%8B%A4%EC%8A%B5#:~:text=%EA%B9%83%ED%97%88%EB%B8%8C%20%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C%20%EB%A7%81%ED%81%AC-,pretrained%20%EB%AA%A8%EB%8D%B8,-%ED%99%98%EA%B2%BD%ED%8C%8C%EC%9D%BC%20%EB%AA%A8%EB%8D%B8)

[환경파일 모델](https://github.com/opencv/opencv_extra/blob/master/testdata/dnn/faster_rcnn_resnet50_coco_2018_01_28.pbtxt)

In [ ]:
!mkdir ./pretrained
# pretrained 모델
!wget -O ./pretrained/faster_rcnn_resnet50_coco_2018_01_28.tar.gz http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
# config 파일
!wget -O ./pretrained/config_graph.pbtxt https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/faster_rcnn_resnet50_coco_2018_01_28.pbtxt

# 압축파일 풀기
!tar -xvf ./pretrained/faster*.tar.gz -C ./pretrained

--2023-11-28 07:08:03--  http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.164.27, 172.217.0.91, 172.217.12.27, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.164.27|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381355771 (364M) [application/x-tar]
Saving to: ‘./pretrained/faster_rcnn_resnet50_coco_2018_01_28.tar.gz’

./pretrained/faster 100%[===================>] 363.69M   328MB/s    in 1.1s    

2023-11-28 07:08:05 (328 MB/s) - ‘./pretrained/faster_rcnn_resnet50_coco_2018_01_28.tar.gz’ saved [381355771/381355771]

--2023-11-28 07:08:05--  https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/faster_rcnn_resnet50_coco_2018_01_28.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.co

체크포인트, 메타 정보 등 다양하게 있지만.. 주로 사용하는 파일은 .pb 로 끝나는 모델과 .pbtxt 인 환경파일이다.  
여기까지 로딩해주자! 로딩시에는 .readNetFromTensorflow 를 이용하면 된다.

In [ ]:
cv_net = cv2.dnn.readNetFromTensorflow('./pretrained/faster_rcnn_resnet50_coco_2018_01_28/frozen_inference_graph.pb','./pretrained/config_graph.pbtxt')

#### ✅ 다음으로는 모델, 이미지가 준비되었을 때 또다르게 필요한 클래스명 지정단계이다.
클래스명 지정은 MS COCO데이터 세트에서 숫자와 해당 클래스 이름을 매핑하기 위해 하나의 변수에다 해당 변수들을 딕셔너리로 넣는 것을 말한다.  
여기서 헷갈릴 수 있는 점이.. tensorflow 에서 훈련된 모델을 OpenCV에서 돌릴 때 매핑되는 클래스 개수가 조금씩 다르다! 아래 정리된 내용을 확인하자.  
  
- Tensorflow Faster RCNN : 0-90까지
- Tensorflow SSD : 1-91까지
- Tensorflow Mask RCNN : 0-90까지
- Darknet YOLO: 0-79까지

현재는 Faster RCNN에 대한 구현을 할 것이므로 0-90까지 91개로 매핑된 딕셔너리가 필요하다.

In [ ]:
# OpenCV Tensorflow Faster-RCNN용 -> 91개
labels_to_names_0 = {0:'person',1:'bicycle',2:'car',3:'motorcycle',4:'airplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',
                    10:'fire hydrant',11:'street sign',12:'stop sign',13:'parking meter',14:'bench',15:'bird',16:'cat',17:'dog',18:'horse',19:'sheep',
                    20:'cow',21:'elephant',22:'bear',23:'zebra',24:'giraffe',25:'hat',26:'backpack',27:'umbrella',28:'shoe',29:'eye glasses',
                    30:'handbag',31:'tie',32:'suitcase',33:'frisbee',34:'skis',35:'snowboard',36:'sports ball',37:'kite',38:'baseball bat',39:'baseball glove',
                    40:'skateboard',41:'surfboard',42:'tennis racket',43:'bottle',44:'plate',45:'wine glass',46:'cup',47:'fork',48:'knife',49:'spoon',
                    50:'bowl',51:'banana',52:'apple',53:'sandwich',54:'orange',55:'broccoli',56:'carrot',57:'hot dog',58:'pizza',59:'donut',
                    60:'cake',61:'chair',62:'couch',63:'potted plant',64:'bed',65:'mirror',66:'dining table',67:'window',68:'desk',69:'toilet',
                    70:'door',71:'tv',72:'laptop',73:'mouse',74:'remote',75:'keyboard',76:'cell phone',77:'microwave',78:'oven',79:'toaster',
                    80:'sink',81:'refrigerator',82:'blender',83:'book',84:'clock',85:'vase',86:'scissors',87:'teddy bear',88:'hair drier',89:'toothbrush',
                    90:'hair brush'}

### ✅ Inference
본격적으로 예측을 만드는 코드를 보자! 코드는 2번에 잘라서 보도록 해보자.

In [ ]:
import time
# 모델, 이미지, threshold
def get_detected_img(cv_net, img_array, score_threshold, use_copied_array=True, is_print=True):

    rows = img_array.shape[0]
    cols = img_array.shape[1]

    draw_img = None
    if use_copied_array:
        draw_img = img_array.copy()
    else:
        draw_img = img_array

    cv_net.setInput(cv2.dnn.blobFromImage(img_array, swapRB=True, crop=False))

    start = time.time()
    cv_out = cv_net.forward()

    green_color=(0, 255, 0)
    red_color=(0, 0, 255)

    # detected 된 object들을 iteration 하면서 정보 추출
    for detection in cv_out[0,0,:,:]:
        score = float(detection[2])
        class_id = int(detection[1])
        # detected된 object들의 score가 함수 인자로 들어온 score_threshold 이상만 추출
        if score > score_threshold:
            # detected된 object들은 scale된 기준으로 예측되었으므로 다시 원본 이미지 비율로 계산
            left = detection[3] * cols
            top = detection[4] * rows
            right = detection[5] * cols
            bottom = detection[6] * rows
            # labels_to_names 딕셔너리로 class_id값을 클래스명으로 변경. opencv에서는 class_id + 1로 매핑해야함.
            caption = "{}: {:.4f}".format(labels_to_names_0[class_id], score)
            print(caption)
            #cv2.rectangle()은 인자로 들어온 draw_img에 사각형을 그림. 위치 인자는 반드시 정수형.
            cv2.rectangle(draw_img, (int(left), int(top)), (int(right), int(bottom)), color=green_color, thickness=3)
            cv2.putText(draw_img, caption, (int(left), int(top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1)
    if is_print:
        print('Detection 수행시간:',round(time.time() - start, 2),"초")

    return draw_img

In [ ]:
# tensorflow inference 모델 로딩
cv_net = cv2.dnn.readNetFromTensorflow('./pretrained/faster_rcnn_resnet50_coco_2018_01_28/frozen_inference_graph.pb',
                                     './pretrained/config_graph.pbtxt')

## Faster R-CNN 영상처리 실습  
이미지가 아닌 영상을 처리해본다. 그러나 로직은 크게 다르지 않고 프레임마다 object detection을 수행한 것을 이어붙이기만 하면 된다.

### 1) video capture와 videoWriter 설정   
VideoCapture 을 사용하면 프레임별로 캡쳐할 수 있는 상태가 되고, 프레임의 크기와 fps 를 설정할 수 있다. 또한 VideoWriter 을 통해 영상을 쓸 수 있는데, 이때 인자로 전달해줄 몇가지 정보들이 필요하니 알아보자.

In [ ]:
video_input_path = '/content/drive/MyDrive/study/vision/video.mp4'

cap = cv2.VideoCapture(video_input_path)
# 프레임별로 캡쳐할 수 있는 상태
frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # cap 후 get 을 하면 프레임카운트가 가능
print('총 Frame 갯수:', frame_cnt)

총 Frame 갯수: 56


이제 비디오캡쳐와 라이터 두 가지를 설정해주자. 캡쳐는 그대로 수행하고, Writer 설정을 위해선 codec, size, fps 세가지 정보가 필요하다.

In [ ]:
video_input_path = '/content/drive/MyDrive/study/vision/video.mp4'
video_output_path = '/content/drive/MyDrive/study/vision/video_cv01.mp4'

cap = cv2.VideoCapture(video_input_path)

codec = cv2.VideoWriter_fourcc(*'XVID') # -> video writer 을 어떤 포맷으로? 코덱 설정

# 비디오 사이즈와 비디오 fps 설정
vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
vid_fps = cap.get(cv2.CAP_PROP_FPS)
# writer 에 입력과 동일하게 인자 전달
vid_writer = cv2.VideoWriter(video_output_path, codec, vid_fps, vid_size)

frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('총 Frame 갯수:', frame_cnt)

총 Frame 갯수: 56


이제 cap 과 vid_writer 가 어떻게 이용되는지 주의깊게 보면된다.

### 2) Object Detection 수행 (프레임별로)

In [ ]:
# bounding box의 테두리와 caption 글자색 지정
green_color=(0, 255, 0)
red_color=(0, 0, 255)

# 영상 프레임을 돌면서 프레임이 없을 때까지 object detection
while True:

    hasFrame, img_frame = cap.read() # -> cap.read() 를 하게되면 다음에 프레임이 있는지와, 현재 프레임정보 반환

    if not hasFrame:
        print('더 이상 처리할 frame이 없습니다.')
        break

    rows = img_frame.shape[0]
    cols = img_frame.shape[1]
    # 원본 이미지 배열 BGR을 RGB로 변환하여 배열 입력
    cv_net.setInput(cv2.dnn.blobFromImage(img_frame,  swapRB=True, crop=False))

    start= time.time()
    # Object Detection 수행하여 결과를 cv_out으로 반환
    cv_out = cv_net.forward()
    frame_index = 0
    # detected 된 object들을 iteration 하면서 정보 추출
    for detection in cv_out[0,0,:,:]:
        score = float(detection[2])
        class_id = int(detection[1])
        # detected된 object들의 score가 0.5 이상만 추출
        if score > 0.5:
            # detected된 object들은 scale된 기준으로 예측되었으므로 다시 원본 이미지 비율로 계산
            left = detection[3] * cols
            top = detection[4] * rows
            right = detection[5] * cols
            bottom = detection[6] * rows
            # labels_to_names_0딕셔너리로 class_id값을 클래스명으로 변경.
            caption = "{}: {:.4f}".format(labels_to_names_0[class_id], score)
            #print(class_id, caption)
            #cv2.rectangle()은 인자로 들어온 draw_img에 사각형을 그림. 위치 인자는 반드시 정수형.
            cv2.rectangle(img_frame, (int(left), int(top)), (int(right), int(bottom)), color=green_color, thickness=2)
            cv2.putText(img_frame, caption, (int(left), int(top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1)
    print('Detection 수행 시간:', round(time.time()-start, 2),'초')
    vid_writer.write(img_frame)
# end of while loop

vid_writer.release()
cap.release()

Detection 수행 시간: 6.71 초
Detection 수행 시간: 7.5 초
Detection 수행 시간: 7.66 초
Detection 수행 시간: 6.14 초
Detection 수행 시간: 8.59 초
Detection 수행 시간: 6.09 초
Detection 수행 시간: 8.17 초
Detection 수행 시간: 11.35 초
Detection 수행 시간: 6.89 초
Detection 수행 시간: 8.06 초
Detection 수행 시간: 6.1 초
Detection 수행 시간: 8.21 초
Detection 수행 시간: 7.54 초
Detection 수행 시간: 6.99 초
Detection 수행 시간: 7.88 초
Detection 수행 시간: 6.06 초
Detection 수행 시간: 8.44 초
Detection 수행 시간: 6.36 초
Detection 수행 시간: 6.91 초
Detection 수행 시간: 7.82 초
Detection 수행 시간: 6.04 초
Detection 수행 시간: 8.23 초
Detection 수행 시간: 6.6 초
Detection 수행 시간: 6.69 초
Detection 수행 시간: 8.09 초
Detection 수행 시간: 6.15 초
Detection 수행 시간: 8.09 초
Detection 수행 시간: 6.78 초
Detection 수행 시간: 6.47 초
Detection 수행 시간: 8.22 초
Detection 수행 시간: 6.09 초
Detection 수행 시간: 7.88 초
Detection 수행 시간: 6.97 초
Detection 수행 시간: 6.52 초
Detection 수행 시간: 8.3 초
Detection 수행 시간: 6.14 초
Detection 수행 시간: 8.04 초
Detection 수행 시간: 6.82 초
Detection 수행 시간: 6.55 초
Detection 수행 시간: 8.21 초
Detection 수행 시간: 6.11 초
Detection 수행 시간: 7.

- cap.read() 는 다음 프레임이 있는지 True or False 정보와, 현재 프레임을 반환하는 중요한 메서드다. 만약 다음 프레임이 없으면 반복문을 빠져나오는 코드를 넣어주자.
  
- 다음은 이미지에서 Object Detection 과 거의 같다. 이미지를 네트웍에 입력하고, forward() 를 수행하며.. 각 프레임에서 object detection 을 수행해서 네모박스를 씌우고 텍스트를 넣으면 된다.

- 이때 하나의 이미지 프레임 연산이 끝나면 vid_writer.write() 를 통해 비디오를 써주자.

- 모두 끝나면 vid_writer 와 cap 을 .release() 를 통해 종료시킨다.

- 영상을 실행시켜보면 프레임별로 열심히.. object detection 된 것을 확인할 수 있다!

### 3) Video Detection 함수 생성  
이제 기본 틀이 되는 함수 형태로 생성해보자.

In [ ]:
def do_detected_video(cv_net, input_path, output_path, score_threshold, is_print):

    cap = cv2.VideoCapture(input_path)

    codec = cv2.VideoWriter_fourcc(*'XVID')

    vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_fps = cap.get(cv2.CAP_PROP_FPS)

    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size)

    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt)

    green_color=(0, 255, 0)
    red_color=(0, 0, 255)
    while True:
        hasFrame, img_frame = cap.read()
        if not hasFrame:
            print('더 이상 처리할 frame이 없습니다.')
            break

        img_frame = get_detected_img(cv_net, img_frame, score_threshold=score_threshold, use_copied_array=False, is_print=is_print)

        vid_writer.write(img_frame)
    # end of while loop

    vid_writer.release()
    cap.release()

함수 실행

In [ ]:
do_detected_video(cv_net, '/content/data/Jonh_Wick_small.mp4', './data/John_Wick_small_02.mp4', 0.2, False)